In [30]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from difflib import SequenceMatcher
from bs4 import BeautifulSoup
############################################################
import os, csv
import time



class search_save_CSV:
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3")
        options.add_argument('Accept-Charset=utf-8')
        
        self.driver = webdriver.Chrome("./chromedriver.exe", options=options)
        self.driver.implicitly_wait(10)
        
    def getKeyword_andOpen(self, search_text):
        self.driver.get("https://m.map.naver.com/")
        
        time.sleep(1)
        
        self.search_ = search_text
        self.driver.find_element_by_class_name("Nsearch._searchKeywordView._searchGuide").click()
        
        self.driver.find_element_by_class_name("Nbox_input_text._search_input").send_keys(self.search_)
        self.driver.find_element_by_class_name("_formSearch").submit()
        
        
    def search_and_save(self):
        searchList = self.driver.find_elements_by_class_name("_item._lazyImgContainer")
        choicePer, choiceIndex = 0, -1
        for i, element in enumerate(searchList):
            # print(i, ":", element.get_attribute("data-title"))    
            ratio = self.getMatchPercent(element.get_attribute("data-title"), self.search_)
            print("{0:.1f}%".format(ratio))
            
            if ratio < 50:
                pass
            else:
                if choicePer < ratio:
                    choicePer = ratio
                    choiceIndex = i
        if choiceIndex == -1:
            print("일치결과 없음")
            return False
        else:
            print("선택 index {0} : {1} 과의 일치율 {2:.1f}".format(choiceIndex, self.search_, choicePer))
            self.index = choiceIndex
            return True
        
    def get_menu_list(self): 
        elements = self.driver.find_elements_by_class_name("sp_map.btn_price._linkMenu")
        try:
            elements[self.index].click()
        except Exception:
            "가격정보 없음"
            return False
        time.sleep(5)
        
        temp = str(self.driver.current_url)
        temp = temp[:-4] + 'menu/list'
        print(temp)
        self.driver.get(temp)
        
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        elements = soup.find_all("li", class_="_1jXsR")
        menuName = soup.find_all("span", class_="_1zr5s")
        menuPrice = soup.find_all("div", class_="_1cQ4a")

        for i, menu in enumerate(menuName):
            print(i, menu.text, ":", menuPrice[i].text)
        self.name = menuName
        self.price = menuPrice
        return True
        
    def save_to_csv(self, pk):
        saveFileName = os.path.abspath("../data/price.csv")
        if not os.path.exists(saveFileName):
            file = open(saveFileName, "w", newline="", encoding='utf-8')
            newWriter = csv.writer(file)
            newWriter.writerow(["관리번호", "메뉴이름", "가격"])
            file.close()
        file = open(saveFileName, "a", newline="", encoding='utf-8')
        writer = csv.writer(file)
        for i, menu in enumerate(self.name):
            row = [str(pk), menu.text, self.price[i].text]
            writer.writerow(row)
        file.close()      
        
        
    def getMatchPercent(self, one, two):
        s = SequenceMatcher(None, one, two)
        return s.ratio() * 100
    
    def close_driver(self):
        self.driver.quit()

In [9]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("coffee-price").setMaster("local[*]")
sc = SparkContext(conf=conf).getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=coffee-price, master=local[*]) created by __init__ at <ipython-input-2-f3cc6e999545>:4 

In [10]:
import os

data_path = os.path.join("..", "data", "data.csv")
raw_data = sc.textFile(data_path)
raw_data_header = raw_data.take(1)[0]

In [11]:
headerList = str(raw_data_header).split(",") # 관리번호 = PK
need_column = ['관리번호', '영업상태구분코드', '영업상태명', '소재지전화',\
        '소재지면적','소재지전체주소', '도로명전체주소', '사업장명', '최종수정시점',\
        '업태구분명', '좌표정보(X)', '좌표정보(Y)', '시설총규모', '홈페이지']
need_column_index = []
column_list = []
for i, value in enumerate(headerList):
    if str(value) in need_column:
        need_column_index.append(i)
        column_list.append(str(value))

In [12]:
data = raw_data.filter(lambda line: line!=raw_data_header) \
    .map(lambda line: line.split(",")) \
    .filter(lambda tokens: tokens[24] == '커피숍' and tokens[7] =='영업/정상') \
    .map(lambda tokens: [tokens[i] for i in need_column_index])

In [31]:
elements = data.take(100000)
dongList = list()
search = search_save_CSV()
for i, element in enumerate(elements):
    print(i, "번째.")
#     if i%10 == 9:
#         search.close_driver()
#         # time.sleep(1)
#         search = search_save_CSV()
    for i in range(len(column_list)):
        print("[%s]" % i, column_list[i], ":", element[i])
    continue_list = \
    ["스타벅스", "이디야", "투썸", "투썸플레이스", "엔젤리너스", "커피빈", "구스토", "달콤커피", "메가커피", \
    "바나프레소", "빽다방", "셀렉토", "요거프레소", "주커피", "드롭탑", "띠아모", "카페베네", "봄봄", \
    "커핀그루나루", "커피마마", "커피에 반하다", "탐앤탐스", "폴 바셋", "할리스"]
    
    name_list = str(element[7]).split()
    flag = False
    for comp in name_list:
        for li in continue_list:
            if li in comp:
                flag = True
                break
        if flag:
            break
    if flag: # 프랜차이즈의 경우는 넘어간다.
        continue
    
    address = str(element[5]).split()
    try:
        dongList.append(address[2])
        print("************************************************")
        search_keyword = address[2] + " " + element[7]
        search.getKeyword_andOpen(search_keyword)
        flag_ = search.search_and_save()
        if flag_:
            if search.get_menu_list():
                time.sleep(1)
                search.save_to_csv(element[0])
    except Exception:
        continue
search.close_driver()

0 번째.
[0] 관리번호 : 3000000-104-2007-00059
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 신문로1가 58-12번지 나동 지하1층
[6] 도로명전체주소 : 서울특별시 종로구 새문안로3길 11 (신문로1가 나동 지하1층)
[7] 사업장명 : 커피온더힐
[8] 최종수정시점 : 20070827100301
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 197490.303085467
[11] 좌표정보(Y) : 452117.063960596
[12] 시설총규모 : 20.2
[13] 홈페이지 : None
************************************************
일치결과 없음
1 번째.
[0] 관리번호 : 3000000-104-2007-00062
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 02  22653690
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 종로3가 124번지 (지상1층)
[6] 도로명전체주소 : 서울특별시 종로구 종로 126 (종로3가 (지상1층))
[7] 사업장명 : 이디야커피
[8] 최종수정시점 : 20160624150745
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 199182.655264501
[11] 좌표정보(Y) : 451979.234244984
[12] 시설총규모 : 60
[13] 홈페이지 : None
2 번째.
[0] 관리번호 : 3000000-104-2007-00064
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 평동 85-15번지 (지상1층)
[6] 도로명전체주소 : 서울특별시 종로구 경교장길 8 (평동 (지상1층))
[7] 사업장명 : 그라찌에 중앙대

75.0%
선택 index 0 : 중학동 고디바케이트윈타워 과의 일치율 75.0
https://m.place.naver.com/restaurant/1401032324/menu/list
0 다크초콜릿 데카당스 : 7,500원
1 컵 아이스크림 : 5,500원
2 아메리카노 : 4,000원
3 카페라떼 : 4,800원
4 다크탄자니아75%초콜렉사 : 7,500원
17 번째.
[0] 관리번호 : 3000000-104-2018-00104
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 18.00
[5] 소재지전체주소 : 서울특별시 종로구 체부동 209번지
[6] 도로명전체주소 : 서울특별시 종로구 자하문로1길 44  1층 (체부동)
[7] 사업장명 : 너만 모르는 밀크티
[8] 최종수정시점 : 20190627093328
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 197277.961691152
[11] 좌표정보(Y) : 452744.136825054
[12] 시설총규모 : 18
[13] 홈페이지 : None
************************************************
72.7%
선택 index 0 : 체부동 너만 모르는 밀크티 과의 일치율 72.7
https://m.place.naver.com/restaurant/1987513835/menu/list
18 번째.
[0] 관리번호 : 3000000-104-2018-00111
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 계동 2-49번지
[6] 도로명전체주소 : 서울특별시 종로구 계동길 112  1층 (계동)
[7] 사업장명 : 부스갤러리(Booth gallery)
[8] 최종수정시점 : 20180627140744
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 198774.65039998

일치결과 없음
33 번째.
[0] 관리번호 : 3000000-104-2012-00177
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 14.28
[5] 소재지전체주소 : 서울특별시 종로구 당주동 160번지
[6] 도로명전체주소 : 서울특별시 종로구 새문안로5길 13  지하1층 5호 (당주동)
[7] 사업장명 : 커피붕붕 커피볶는집
[8] 최종수정시점 : 20191014134350
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 197673.919714838
[11] 좌표정보(Y) : 452123.434528869
[12] 시설총규모 : 14.28
[13] 홈페이지 : None
************************************************
83.3%
선택 index 0 : 당주동 커피붕붕 커피볶는집 과의 일치율 83.3
https://m.map.naver.com/search2/search.nhn?query=%EB%8B%B9%EC%A3%BC%EB%8F%99%20%EC%BB%A4%ED%94%BC%EB%B6%95%EB%B6%95%20%EC%BB%A4%ED%94%BC%EB%B3%B6%EB%8A%94%EC%A7%91&sm=hty&stylmenu/list
34 번째.
[0] 관리번호 : 3000000-104-2019-00231
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 100.00
[5] 소재지전체주소 : 서울특별시 종로구 명륜4가 198-3번지 성우빌딩
[6] 도로명전체주소 : 서울특별시 종로구 창경궁로 227  성우빌딩 1층 (명륜4가)
[7] 사업장명 : 티티야 카페
[8] 최종수정시점 : 20191210153733
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 199730.620290942
[11] 좌표정보(Y) : 453348.895548068
[12] 시설총규모 : 100
[13]

50.0%
0.0%
선택 index 0 : 체부동 공유까페 과의 일치율 50.0
https://m.place.naver.com/restaurant/1220281911/menu/list
0 아메리카노 : 4,600원
1 카페라떼 : 5,300원
2 흑당라떼 : 5,600원
3 카라멜마끼아또 : 6,200원
4 솜사탕 라떼 : 6,800원
49 번째.
[0] 관리번호 : 3000000-104-2019-00173
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 84.18
[5] 소재지전체주소 : 서울특별시 종로구 명륜2가 209-2번지
[6] 도로명전체주소 : 서울특별시 종로구 성균관로 8  1층 (명륜2가)
[7] 사업장명 : 이디야
[8] 최종수정시점 : 20190902114837
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 199783.918141466
[11] 좌표정보(Y) : 453483.532108771
[12] 시설총규모 : 84.18
[13] 홈페이지 : None
50 번째.
[0] 관리번호 : 3000000-104-2019-00177
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 15.40
[5] 소재지전체주소 : 서울특별시 종로구 계동 140-56번지
[6] 도로명전체주소 : 서울특별시 종로구 창덕궁1길 32  1층 (계동)
[7] 사업장명 : 카페단밀
[8] 최종수정시점 : 20190906152321
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 198824.349286311
[11] 좌표정보(Y) : 453013.574565871
[12] 시설총규모 : 15.4
[13] 홈페이지 : None
************************************************
72.7%
선택 index 0 : 계동 카페단밀 과의 일치율 72.7
https://m.place.naver.co

78.3%
선택 index 0 : 종로2가 쥬스식스 종각역점 과의 일치율 78.3
67 번째.
[0] 관리번호 : 3000000-104-2007-00024
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 02 7625221
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 명륜2가 225번지 지상1층
[6] 도로명전체주소 : 서울특별시 종로구 성균관로 9-1 (명륜2가 지상1층)
[7] 사업장명 : 띵똥와플
[8] 최종수정시점 : 20140129160328
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 199743.555078365
[11] 좌표정보(Y) : 453491.538549091
[12] 시설총규모 : 12.74
[13] 홈페이지 : None
************************************************
61.5%
선택 index 0 : 명륜2가 띵똥와플 과의 일치율 61.5
https://m.place.naver.com/restaurant/35438754/menu/list
0 와플 : 1,200~2,700원
68 번째.
[0] 관리번호 : 3000000-104-2017-00016
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 당주동 100번지
[6] 도로명전체주소 : 서울특별시 종로구 세종대로23길 54  지하1층층 119호 (당주동)
[7] 사업장명 : 모스트커피
[8] 최종수정시점 : 20170126165637
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 197573.030749573
[11] 좌표정보(Y) : 452175.335781858
[12] 시설총규모 : 23.02
[13] 홈페이지 : None
************************************************
71.4%
선택 index 0 :

...

일치결과 없음
99 번째.
[0] 관리번호 : 3000000-104-2019-00074
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 66.12
[5] 소재지전체주소 : 서울특별시 종로구 충신동 16-1번지
[6] 도로명전체주소 : 서울특별시 종로구 율곡로23길 18-8  2층 (충신동)
[7] 사업장명 : cafe 다-품
[8] 최종수정시점 : 20190404091841
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 200535.146036777
[11] 좌표정보(Y) : 452560.351172935
[12] 시설총규모 : 66.12
[13] 홈페이지 : None
************************************************
일치결과 없음
100 번째.
[0] 관리번호 : 3000000-104-2019-00131
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 000203915949
[4] 소재지면적 : 23.10
[5] 소재지전체주소 : 서울특별시 종로구 구기동 88-40번지
[6] 도로명전체주소 : 서울특별시 종로구 진흥로 445  2층 (구기동)
[7] 사업장명 : J카페
[8] 최종수정시점 : 20190625120036
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 196231.817423526
[11] 좌표정보(Y) : 456156.199742205
[12] 시설총규모 : 23.1
[13] 홈페이지 : None
************************************************
일치결과 없음
101 번째.
[0] 관리번호 : 3000000-104-2019-00121
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 11.00
[5] 소재지전체주소 : 서울특별시 종로구 통의동 55번지
[6] 도로명전체주소 : 서울특별

115 번째.
[0] 관리번호 : 3000000-104-2019-00019
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 34.46
[5] 소재지전체주소 : 서울특별시 종로구 연건동 28-21번지
[6] 도로명전체주소 : 서울특별시 종로구 대학로 101  첨단외래센터지하1층 (연건동)
[7] 사업장명 : 잠바주스 서울대병원점
[8] 최종수정시점 : 20191125145524
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 199824.095974723
[11] 좌표정보(Y) : 452984.360451452
[12] 시설총규모 : 34.46
[13] 홈페이지 : None
************************************************
일치결과 없음
116 번째.
[0] 관리번호 : 3000000-104-2019-00020
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 02 7200427
[4] 소재지면적 : 34.44
[5] 소재지전체주소 : 서울특별시 종로구 궁정동 29-2번지 1층
[6] 도로명전체주소 : 서울특별시 종로구 효자로 107  1층 (궁정동)
[7] 사업장명 : 스탠딩커피
[8] 최종수정시점 : 20200313174438
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 197394.332320066
[11] 좌표정보(Y) : 453482.632325566
[12] 시설총규모 : 34.44
[13] 홈페이지 : None
************************************************
52.6%
선택 index 0 : 궁정동 스탠딩커피 과의 일치율 52.6
117 번째.
[0] 관리번호 : 3000000-104-2019-00029
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 74.91
[5] 소재지전체주소 : 

일치결과 없음
136 번째.
[0] 관리번호 : 3000000-104-2009-00013
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 명륜2가 114-2번지 지상1층
[6] 도로명전체주소 : None
[7] 사업장명 : 홀릭
[8] 최종수정시점 : 20090303173333
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 199706.120891468
[11] 좌표정보(Y) : 453592.054492731
[12] 시설총규모 : 7.4
[13] 홈페이지 : None
************************************************
일치결과 없음
137 번째.
[0] 관리번호 : 3000000-104-2009-00016
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 견지동 110번지 종로1가대성스카이렉스 111호
[6] 도로명전체주소 : 서울특별시 종로구 삼봉로 95 (견지동 종로1가대성스카이렉스 111호)
[7] 사업장명 : 뉴욕 앤 핫도그
[8] 최종수정시점 : 20090311163949
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 198386.829870878
[11] 좌표정보(Y) : 452237.303784872
[12] 시설총규모 : 44.65
[13] 홈페이지 : None
************************************************
일치결과 없음
138 번째.
[0] 관리번호 : 3000000-104-2009-00038
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 02 3790056
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 구기동 166-3번지 (지상1층)
[6] 도로명

일치결과 없음
156 번째.
[0] 관리번호 : 3000000-104-2015-00050
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 창신동 436-73번지 1층
[6] 도로명전체주소 : 서울특별시 종로구 청계천로 341  1층 (창신동)
[7] 사업장명 : 컵퍼(CUPPER)
[8] 최종수정시점 : 20150605151501
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 201302.883783666
[11] 좌표정보(Y) : 451982.388503793
[12] 시설총규모 : 38
[13] 홈페이지 : None
************************************************
일치결과 없음
157 번째.
[0] 관리번호 : 3000000-104-2014-00114
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 서울특별시 종로구 인의동 48-2번지 효성주얼리시틸 지하1층 1028호
[6] 도로명전체주소 : 서울특별시 종로구 종로 183  지하1층 1028호 (인의동  효성주얼리시티)
[7] 사업장명 : 카페이너스
[8] 최종수정시점 : 20140613114820
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 199812.609633255
[11] 좌표정보(Y) : 452132.648343993
[12] 시설총규모 : 11.97
[13] 홈페이지 : None
************************************************
일치결과 없음
158 번째.
[0] 관리번호 : 3000000-104-2013-00157
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 070 86280476
[4] 소재지면적 : None
[5] 소재지전체주소 : 서

KeyboardInterrupt: 